https://github.com/Ioana-P/pca_and_clustering_for_edu_purposes/blob/master/newsgroups_LSA.ipynb
https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/  
https://stackabuse.com/python-for-nlp-topic-modeling/

BERTopic https://pypi.org/project/bertopic/, https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.transform bertopics needs Python > 3.7

In [ ]:
import os
path = "C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/nlp"
os.chdir(path)

import pandas as pd
import numpy as np

import fasttext

import Import_and_clean_data as ic
import WCS_Clustering_MainFunction as cmf
import SlimWeightedVecData as swd
import OutputWarranty as ow
import WarrantyCluster as wc


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation  
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import TruncatedSVD
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install bertopic
#!pip install torch

In [ ]:
# showing multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df_path = "C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/data/df_ny_gl_system_1_azure.csv"
stopword_path = "C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/nlp/danska-stopwords.csv"
fasttext_link = "cc.da.300.bin"

In [ ]:
# here Text-variable is created for analysis 
# # there are some rows without any system assigned
# df = ic.ImportAndCleanCSV(df_path, datenc = "utf-8", text_multi_var = 2, text_var = "Beskrivelse")
# df.shape
# df.drop_duplicates(keep = False, inplace = True)
# df.shape

In [ ]:
from bertopic import BERTopic
#from sklearn.datasets import fetch_20newsgroups

In [ ]:

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

topic_model = BERTopic()
topics, _ = topic_model.fit_transform(docs)

### Creating Category-Variable for Topic Modeling

In [ ]:
def ImportAndCleanCSV(datapath, datenc, text_var, new_col):
    # Import and clean data
    wcsdata = pd.read_csv(datapath, encoding = datenc, sep = ";")
    wcsdata.fillna('', inplace=True)

    # To datetime
    wcsdata["Meddelelsesdato"] = pd.to_datetime(wcsdata['Meddelelsesdato'], format='%Y-%m-%d')
    wcsdata["Afslutningsdato"] = pd.to_datetime(wcsdata['Afslutningsdato'], format='%Y-%m-%d')

    # Combine DefectHeading, DefectDescription and DefectCodeName
    wcsdata[text_var] = wcsdata[text_var].astype(str)
    wcsdata["Subsystem"] = wcsdata["Subsystem"].astype(str)
    wcsdata[new_col] = wcsdata[text_var] + " " + wcsdata["Subsystem"] + " " + \
    wcsdata["System_text"] + " " + wcsdata["Subsys_text"] 

    return wcsdata

In [ ]:
df = ImportAndCleanCSV(df_path, datenc = "utf-8", text_var = "System", new_col = "Category")
df.shape
df

In [ ]:
df.groupby("Category")["Category"].count()

In [ ]:
df.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/output/category.csv',
                         decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

In [ ]:
# Spliting the data to two parts by "Landskod". Creating the new column "Category" from "Text".
df_et = df[df["Landskod"] == "ET"]
# df_et.shape
# df_et.loc[df["Landskod"] == "ET", "Category"] = df["Text"]
df_et.shape
df_et

In [ ]:
df_et.groupby("Category")["Category"].count()

In [ ]:
# import seaborn as sns
df_et["cat"] = df_et["System"] + df_et["Subsystem"]
# sns.distplot(df_et.groupby("System").count())
sns.set(rc={'figure.figsize':(11.7,8.27)})
g = sns.catplot(x = "cat", kind = "count", data = df_et, height=8.27, aspect=11.7/8.27)
g.set_xticklabels(rotation=30)

In [ ]:
np.where(pd.isna(df_et["Category"]))
df_et[df_et["Category"] == " "].index

In [ ]:
# creating index for each category
df_et['idx'] = df_et.groupby("Category").ngroup()
#df_et[df_et["idx"] == 4]

In [ ]:
df_et = df_et.sort_values("idx")
df_et_drop = df_et.drop(df_et[df_et["idx"] == 0].index)
df_et_drop.shape

In [ ]:
# df_et.groupby("Category").Category.count()
# df_et.groupby("idx").Category.count()

cat_idx_df =df_et[["Category", "idx"]].sort_values("idx")
cat_idx_df.shape
cat_idx_df.head(30)
cat_idx_df.tail()

In [ ]:
df_et.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/output/category_et.csv',
                         decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

### Different landskod giving us different format of faults

Lack of data for landskod == 86.

In [ ]:
#df["Meddelelse"]
# df[df["Landskod"] == "ET"]["Meddelelse"]
# df[df["Landskod"] == "86"]["Meddelelse"]

In [ ]:
# replacing column "Text" with our analysis target
df_et_drop["Text"] = df_et_drop["Beskrivelse"] + " " + df_et_drop["Langtekst"]
df_et_drop

In [ ]:
# summarization
# df_et['Text_Cat'] = df_et.groupby('Category')['Text'].transform(lambda x: ','.join(x))
# df_et

In [ ]:
# #text_cat_df = df_et[["Category", "idx", "Text_Cat"]].drop_duplicates(keep = False)
# text_cat_df = pd.concat(df_et["Category"], df_et["idx"], df_et["Text_Cat"]).unique()
# text_cat_df.shape

In [ ]:
# cleaning Text

import re
import numpy as np
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import itertools

# Import stopwords
stop_words = pd.read_csv(stopword_path, encoding = "ISO 8859-1", sep = " ")
stopwords_custom = stop_words['stoppord'].values.tolist()
  
stopwords_swe = stopwords.words('swedish')
stopwords_dan = stopwords.words('danish')

stop_words = itertools.chain(stopwords_swe, stopwords_dan, stopwords_custom)
stop_words = list(stop_words)

    
def PreprocessText(document, stopwords):
    # Remove dates
    document = RemoveDates(document)
    
     # Remove IP address
    document = RemoveIP(document)
    
    # Remove all the special characters
    document = re.sub(r'[^\w]', ' ', str(document))
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    tokens = document.split()
    tokens = [word for word in tokens if word not in stop_words]

    document = ' '.join(tokens)
    
    # Stemming words
    stemmer = SnowballStemmer("swedish")
    document = stemmer.stem(document)
    
    stemmer = SnowballStemmer("danish")
    document = stemmer.stem(document)
    
    stemmer = SnowballStemmer("english")
    document = stemmer.stem(document)
    
    # Remove number when the string only consists of digits
    #document = re.sub(r'\d', '', document)    # not working when digits at the begining of strings re.sub(r' [0-9]+', '', document)

    # Remove single letters and short words 
    #document = ' '.join( [w for w in document.split() if len(w)>2] )

    return document

def RemoveDates(doc):
    doc = re.sub("[0-9]{2,4}.[0-9]{2}.[0-9]{2,4}", "", doc)
    return doc

def RemoveIP(doc):
    doc =  re.sub(r'[0-9]+(?:\.[0-9]+){3}', '', doc) 
    return doc


df_et_drop["Text"] = [PreprocessText(t, stop_words) for t in df_et_drop["Text"]]

In [ ]:
df_et_drop.head(2)

## Obtaining data for clustering

The function DataForClustering is applied to the subset of the data (in the following example, the subset of the data is the one which system = 1.   

The output of the function returns weighted sentence vectors, and index created for each row.

In [ ]:
df_et_clean = swd.DataForClustering(dat = df_et_drop, group = True, 
                                      sort_var_1 = 'idx', sort_var_2 = 'Meddelelsesdato',
                                      textcol = 'Text',
                                      stopwordpath = stopword_path, stopenc = "ISO 8859-1",
                                      fasttext_link = fasttext_link)

Using a small sample to test clustering algorithms.

In [ ]:
df_et_clean.head(5)

## Topic Modeling

## Latent Semantic Analysis

Latent Semantic Analysis (LSA) is a popular, dimensionality-reduction techniques that follows the same method as Singular Value Decomposition.

This is effectively just a truncated singular value decomposition of a (very high-rank and sparse) document-term matrix, with only the  r= n_topics largest singular values preserved.

LSA unsupervised, rests on two points:  
1. The ditributional hypothesis, which states that words with similar meanings appear frequently together.  
2. Singular Value Decomposition SVD.

The words weight is focused in this kind of analysis.

### Latent Dirichlet Allocation

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

In [ ]:
lda = LatentDirichletAllocation(max_iter=5,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0)

In [ ]:
# test 
count_vectorizer = CountVectorizer(max_features=40000)
cat_1 = df_et_clean[df_et_clean["idx"] == 36]
term_matrix = count_vectorizer.fit_transform(cat_1["Text"])
#lda.fit(term_matrix)

In [ ]:
search_params = {'n_components': [5, 10, 15, 20], 'learning_decay': [.5, .7, .9]}
model = GridSearchCV(lda, param_grid=search_params)
model.fit(term_matrix)
# Best model
#best_lda_model = model.best_estimator_
# model parameters
print("Best Model's Params: ", model.best_params_)

In [ ]:
#model_param = output_topics_number(df = df_et_clean, topicModel = lda, textCol = "Text")
count_vectorizer = CountVectorizer(max_features=40000)
category_list = []
best_number_topics = []
# len(df_et_clean["idx"].unique())
for i in (df_et_clean["idx"].unique()): 
    df = df_et_clean[df_et_clean["idx"] == i]

    search_params = {'n_components': [5, 10, 15, 20], 'learning_decay': [.5, .7, .9]}
    model = GridSearchCV(lda, param_grid=search_params)
    term_matrix = count_vectorizer.fit_transform(df["Text"])
    model.fit(term_matrix)
    # Best model
    #best_model = model.best_estimator_
    
    best_number_topics.append((model.best_params_))

In [ ]:
# df_et_clean["idx"].unique()
# df_et_clean["idx"].unique()

In [ ]:
len(best_number_topics)
with open('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/output/best_nr_topics.txt', 
          'w') as f:
    f.write(str(best_number_topics))

### Using Gensim LDA

In [ ]:
#from gensim.corpora.dictionary import Dictionary
from gensim import corpora, models, similarities, utils, matutils
from gensim.models.ldamulticore import LdaMulticore

In [ ]:
cat_1 = df_et_clean[df_et_clean["idx"] == 42]


# convert text to list for convenience
text_list = cat_1["Text"].tolist()

# tokenization. get text in a format that gensim can turn into a dictionary and corpus
texts = [ [word for word in document.split() ] for document in text_list]

# Create a corpus from a list of texts - association word to numeric id
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the model on the corpus.
lda_gensim = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

In [ ]:
#lda_gensim.show_topic(2)  
#lda_gensim.get_topics()
for topic_id in range(lda_gensim.num_topics):
    topk = lda_gensim.show_topic(topic_id, 10)
    topk_words = [ w for w, _ in topk ]
    
    print('{}: {}'.format(topic_id, ' '.join(topk_words)))

In [ ]:
(df_et_clean["idx"].unique())
df_et_clean

In [ ]:
topic_words_list = []

for idx in (df_et_clean["idx"].unique()):  # df_et_clean["idx"].unique()
    
    cat_idx = df_et_clean[df_et_clean["idx"] == idx]
    
    # convert text to list for convenience
    text_list = cat_idx["Text"].tolist()

    # get text in a format that gensim can turn into a dictionary and corpus
    texts = [ [word for word in document.split() ] for document in text_list]

    # Create a corpus from a list of texts
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Train the model on the corpus.
    lda_gensim = models.LdaModel(corpus, id2word=dictionary, num_topics=5)
    
    for topic_id in range(lda_gensim.num_topics):
        topk = lda_gensim.show_topic(topic_id, 10)
        topk_words = [ w for w, _ in topk]
        
        topic_words_list.append('{} {}: {}'.format(idx, topic_id, ' '.join(topk_words)))     

In [ ]:
topic_words_list

In [ ]:
df_et_clean[df_et_clean.idx == 42]

In [ ]:
test_df = pd.DataFrame({"topics": pd.Series(topic_words_list)})
# test_df["Topics"] = re.sub(r"(\d) (\d):", "", str(test_df["topics"]))
# test_df

In [ ]:
test_df_1 = test_df["topics"].str.extract("(\d) (\d)")
topics_df = pd.DataFrame({"Category": test_df_1.loc[:, 0],
              "Topics_index": test_df_1.loc[:, 1],
              "Topics": test_df["topics"]})
topics_df.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/output/topics_res.csv',
                         decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

In [ ]:
# Calling model[corpus] only creates a wrapper around the old corpus document stream – actual 
# conversions are done on-the-fly, during document iteration.
cat_1 = df_et_clean[df_et_clean["idx"] == 42]


# convert text to list for convenience
text_list = cat_1["Text"].tolist()

# get text in a format that gensim can turn into a dictionary and corpus
texts = [ [word for word in document.split() ] for document in text_list]

# Create a corpus from a list of texts
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the model on the corpus.
lda_gensim = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

lda_gensim.get_document_topics(corpus)

In [ ]:
passes= 150
np.random.seed(1) # setting up random seed to get the same results
ldamodel= LdaMulticore(corpus, 
                    id2word=dictionary, 
                    num_topics=5, 
#                   alpha='asymmetric', 
                    chunksize= 4000, 
                    batch= True,
                    minimum_probability=0.001,
                    iterations=350,
                    passes=passes)        

In [ ]:
#ldamodel.show_topics(num_words=25, formatted=False)
for topic_id in range(ldamodel.num_topics):
    topk = lda_gensim.show_topic(topic_id, 10)
    topk_words = [ w for w, _ in topk ]
    
    print('{}: {}'.format(topic_id, ' '.join(topk_words)))

In [ ]:
# Obtaining the main topic for each row:
all_topics = ldamodel.get_document_topics(corpus)
num_docs = len(all_topics)
all_topics_csr= matutils.corpus2csc(all_topics)
all_topics_numpy= all_topics_csr.T.toarray()
major_topic= [np.argmax(arr) for arr in all_topics_numpy]
major_topic
cat_1['major_lda_topic']= major_topic
cat_1

In [ ]:
res = pd.DataFrame()
for idx in (df_et_clean["idx"].unique()): 
    cat_idx = df_et_clean[df_et_clean["idx"] == idx]
    
    # convert text to list for convenience
    text_list = cat_idx["Text"].tolist()

    # get text in a format that gensim can turn into a dictionary and corpus
    texts = [ [word for word in document.split() ] for document in text_list]

    # Create a corpus from a list of texts
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Train the model on the corpus.
    passes= 150
    np.random.seed(1) # setting up random seed to get the same results
    lda_gensim= LdaMulticore(corpus, 
                    id2word=dictionary, 
                    num_topics=5, 
#                   alpha='asymmetric', 
                    chunksize= 4000, 
                    batch= True,
                    minimum_probability=0.001,
                    iterations=350,
                    passes=passes)    
    
    all_topics = lda_gensim.get_document_topics(corpus)
    num_docs = len(all_topics)
    all_topics_csr= matutils.corpus2csc(all_topics)
    all_topics_numpy= all_topics_csr.T.toarray()

    major_topic= [np.argmax(arr) for arr in all_topics_numpy]
    cat_idx['major_lda_topic']= major_topic
    
    res = res.append(cat_idx)

In [ ]:
res.shape
res["idx"].unique()
#df_et_clean["idx"].unique()

In [ ]:
res.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/output/res_withTopics.csv',
                         decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

In [ ]:
res[res.idx == 42]

## Latent Dirichlet Allocation

The LDA is based upon two general assumptions:

- Documents that have similar words usually have the same topic  
- Documents that have groups of words frequently occurring together usually have the same topic.

Mathematically, the above two assumptions can be represented as:

- Documents are probability distributions over latent topics  
- Topics are probability distributions over words

Before we can apply LDA, we need to create vocabulary of all the words in our data. Remember from the previous article, we could do so with the help of a count vectorizer. 

class sklearn.feature_extraction.text.CountVectorizer(*, input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\b\w\w+\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)

ngram_rangetuple (min_n, max_n), default=(1, 1)
The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted. All values of n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable.

## Non-Negative Matrix Factorization (NMF)

In the previous section we used thee count vectorizer, but in this section we will use TFIDF vectorizer since NMF works with TFIDF. We will create a document term matrix with TFIDF. Look at the following script:

In [ ]:
from sklearn.feature_exdef sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))traction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
doc_term_matrix = tfidf_vect.fit_transform(cat_1['Text'].values.astype('U'))

Once the document term matrix is generated, we can create a probability matrix that contains probabilities of all the words in the vocabulary for all the topics. To do so, we can use the NMF class from the sklearn.decomposition module. Look at the following script:

In [ ]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix )

As we did in the previous section, let's randomly get 10 words from our vocabulary:

In [ ]:
import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

In [ ]:
topic_nr = []
topic_con = []
for i,topic in enumerate(nmf.components_):
    topic_nr.append(i)
    topic_con.append([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
len(topic_nr)
len(topic_con )
df_document_topic = pd.DataFrame(list(zip(topic_nr, topic_con)), columns=['topic_nr','topic_con'])
df_document_topic

In [ ]:
len(df_et_clean["idx"].unique())
range(len(df_et_clean["idx"].unique()))

In [ ]:
def topic_model_category(category_idx, doc = df_et_clean["idx"]):
    cat = df_et_clean[doc == category_idx]
    
    tfidf_vect = TfidfVectorizer()
    doc_term_matrix = tfidf_vect.fit_transform(cat["Text"].values.astype('U'))
    nmf = NMF(n_components=5, random_state=42, max_iter = 1000)
    nmf.fit(doc_term_matrix)
    
    topic_nr = []
    topic_con = []
    
    for i,topic in enumerate(nmf.components_):
        topic_nr.append(i)
        topic_con.append([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
        df_document_topic = pd.DataFrame(list(zip(topic_nr, topic_con)), columns=['topic_nr','topic_con'])
    
    return df_document_topic

for cat in range(3): # len(df_et_clean["idx"].unique())
    res = topic_model_category(category_idx = cat)
    res = pd.concat([res])
    #df_document_topic = pd.DataFrame(list(zip(topic_nr, topic_con)), columns=['topic_nr','topic_con'])

# res = topic_model_category(2)
# pd.concat([res])
# res
# res = df_et_clean.groupby("idx").apply(lambda x: topic_model_category(category_idx = x))
# res.shape

In [ ]:
res

In [ ]:
sim_cluster_output_1.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/nlp/output/cosine_sim_cluster_allSystem_defNu2.csv',
                         decimal = ",", header = True, index = False)

In [ ]:
cat = df_et_clean[ df_et_clean["idx"] == 1]
    
tfidf_vect = TfidfVectorizer()
doc_term_matrix = tfidf_vect.fit_transform(cat["Text"].values.astype('U'))
nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix)
    
topic_nr = []
topic_con = []
    
for i,topic in enumerate(nmf.components_):
    topic_nr.append(i)
    topic_con.append([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    df_document_topic = pd.DataFrame(list(zip(topic_nr, topic_con)), columns=['topic_nr','topic_con'])


In [ ]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

## TextBlob Summarization

In [ ]:
from textblob import TextBlob
# from time import sleep
# from textblob.exceptions import NotTranslated 

In [ ]:
text = "Ett   snabbt TÅG, som kör jättefort : över hastighetsgränsen"
blob = TextBlob(text)

text_dan = "En hurtig TOG, der kører meget hurtigt: over hastighedsgrænsen"
blob_dan = TextBlob(text_dan)
print(blob.tags)
for sentence in blob.sentences:
    print(sentence.sentiment)  # returns (polarity, subjectivity)

In [ ]:
def speech_tagging(texts, nlp, allowed_postags=['NN', 'NNP']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
#sim_cluster_output.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/nlp/output/cosine_sim_cluster_output_1.csv',
#                         sep = ";", header = True, encoding = "ISO 8859-1")

sim_cluster_output_1.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/nlp/output/cosine_sim_cluster_allSystem_defNu2.csv',
                         decimal = ",", header = True, index = False)